In [1]:
#-----------------------------------------------------
# SQL_jupyterlab to automate queries and mapping
#
# next update :
# => work with automap_base to exploit ORM
# => look for declarative_base tools
# => build interatives queries with ipywidget
# => check flask and combine with sqlalchemy
#-----------------------------------------------------


from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

# define declarative base
Base = declarative_base()

# connect to immobilier_db
engine = create_engine('sqlite:///immobilier.db', echo = False)

# reflect all tables
Base.metadata.reflect(bind=engine)

# create session
session = Session(engine)

# define object to facilitate sql queries
table = Base.metadata.tables

sell = table["sell"]
features = table["features"]
commune = table["commune"]

In [2]:
from sqlalchemy.sql import func, select, desc, and_, or_, text
import pandas as pd

# execute sql query and return result in a dataframe
def query_to_df(query):
    # return sql statement
    print("Query :", query, "\n")
    
    df = pd.read_sql(query.statement, session.connection())   
    return df

In [3]:
# 1: Number of appartements sold in the first half of 2020

# select only columns without id
sell_col = [col for col in sell.c if not "id" in col.name]

query = (session.query(*sell_col, features.c.type_local)
         .join(features)
         .filter(features.c.type_local == "Appartement")
        )

display(query_to_df(query))

print("Result of query :", query.count())

Query : SELECT sell.date_mutation AS sell_date_mutation, sell.nature_mutation AS sell_nature_mutation, sell.valeur_fonciere AS sell_valeur_fonciere, features.type_local AS features_type_local 
FROM sell JOIN features ON features.id = sell.features_id 
WHERE features.type_local = ? 



,date_mutation,nature_mutation,valeur_fonciere,type_local
0,2020-01-02,Vente,165000.0,Appartement
1,2020-01-08,Vente,720000.0,Appartement
2,2020-01-07,Vente,220900.0,Appartement
3,2020-01-21,Vente,42000.0,Appartement
4,2020-01-07,Vente,262000.0,Appartement
...,...,...,...,...
31370,2020-06-30,Vente,2203720.0,Appartement
31371,2020-06-15,Vente,518300.0,Appartement
31372,2020-06-30,Vente,1290000.0,Appartement
31373,2020-06-30,Vente,726000.0,Appartement


Result of query : 31375


In [4]:
# 2: Proportion of apartments sold by number of rooms

sell_count = (session.query(func.count(sell.c.id))
             .select_from(sell)
              .scalar_subquery()
            )

proportion = (func.round((100.0 * func.count(features.c.id) / sell_count), 2)
              .label("proportion")
             )

pieces_count = (func.count(features.c.id)
                .label("count_by_pieces")
               )

query = (session.query(proportion, 
                       pieces_count, 
                       features.c.nombre_pieces_principales)
         .select_from(sell)
         .join(features)
         .filter(features.c.type_local == "Appartement")
         .group_by(features.c.nombre_pieces_principales)
         .order_by(desc("proportion"))
        )

query_to_df(query)

Query : SELECT round((? * count(features.id)) / (SELECT count(sell.id) AS count_2 
FROM sell), ?) AS proportion, count(features.id) AS count_by_pieces, features.nombre_pieces_principales AS features_nombre_pieces_principales 
FROM sell JOIN features ON features.id = sell.features_id 
WHERE features.type_local = ? GROUP BY features.nombre_pieces_principales ORDER BY proportion DESC 



,proportion,count_by_pieces,nombre_pieces_principales
0,28.63,9781,2
1,26.23,8963,3
2,19.73,6740,1
3,13.06,4461,4
4,3.26,1114,5
5,0.60,204,6
6,0.16,54,7
7,0.09,30,0
8,0.05,17,8
9,0.02,8,9


In [5]:
# 3: List of 10 departements where price per square meter is highest

avg_valeur = (func.avg(sell.c.valeur_fonciere)
              .label("avg_valeur")
             )

avg_surface = (func.avg(features.c.surface_carrez_du_1er_lot)
               .label("avg_surface")
              )

avg_price_meter = (func.round((avg_valeur / avg_surface), 2)
               .label("avg_price_by_square")
              )

query = (session.query(avg_price_meter,
                       avg_valeur,
                       avg_surface,
                       commune.c.code_departement)
         .select_from(sell)
         .join(features)
         .join(commune)
         .group_by(commune.c.code_departement)
         .order_by(desc("avg_price_by_square"))
         .limit(10) 
        )

display(query_to_df(query))

Query : SELECT round(avg(sell.valeur_fonciere) / avg(features.surface_carrez_du_1er_lot), ?) AS avg_price_by_square, avg(sell.valeur_fonciere) AS avg_valeur, avg(features.surface_carrez_du_1er_lot) AS avg_surface, commune.code_departement AS commune_code_departement 
FROM sell JOIN features ON features.id = sell.features_id JOIN commune ON commune.id = sell.commune_id GROUP BY commune.code_departement ORDER BY avg_price_by_square DESC
 LIMIT ? OFFSET ? 



,avg_price_by_square,avg_valeur,avg_surface,code_departement
0,10725.65,613011.711592,57.153823,75
1,9831.09,537771.989663,54.701154,974
2,7349.49,412373.633113,56.109126,92
3,4752.17,275380.987290,57.948529,94
4,4295.69,241007.308576,56.104395,6
5,4254.78,242301.945055,56.948132,58
6,4211.64,251734.890786,59.771203,69
7,4061.41,222278.622650,54.729426,74
8,4015.03,244796.663166,60.970115,78
9,3898.38,220030.894737,56.441579,32


In [6]:
# 4: average price of square meter of a house in "Île-de-france"

code_departement_idf = [75, 77, 78, 91, 92, 93, 94, 95]

query = (session.query(avg_price_meter,
                       avg_valeur,
                       avg_surface,
                       commune.c.code_departement,
                       features.c.type_local
                      )
         .select_from(sell)
         .join(features)
         .join(commune)
         .filter(
             and_(
                 features.c.type_local == "Maison",
                 commune.c.code_departement.in_(code_departement_idf)
             )
         )
        )

display(query_to_df(query))

Query : SELECT round(avg(sell.valeur_fonciere) / avg(features.surface_carrez_du_1er_lot), ?) AS avg_price_by_square, avg(sell.valeur_fonciere) AS avg_valeur, avg(features.surface_carrez_du_1er_lot) AS avg_surface, commune.code_departement AS commune_code_departement, features.type_local AS features_type_local 
FROM sell JOIN features ON features.id = sell.features_id JOIN commune ON commune.id = sell.commune_id 
WHERE features.type_local = ? AND commune.code_departement IN (__[POSTCOMPILE_code_departement_1]) 



,avg_price_by_square,avg_valeur,avg_surface,code_departement,type_local
0,3498.47,319721.194299,91.388923,77,Maison


In [7]:
# 5: List of 10 most expensive appartements

query = (session.query(sell.c.valeur_fonciere,
                       features.c.surface_carrez_du_1er_lot,
                       commune.c.code_departement,
                       features.c.type_local
                      )
         .select_from(sell)
         .join(features)
         .join(commune)
         .filter(features.c.type_local == "Appartement")
         .order_by(desc(sell.c.valeur_fonciere))
         .limit(10)
        )

display(query_to_df(query))

Query : SELECT sell.valeur_fonciere AS sell_valeur_fonciere, features.surface_carrez_du_1er_lot AS features_surface_carrez_du_1er_lot, commune.code_departement AS commune_code_departement, features.type_local AS features_type_local 
FROM sell JOIN features ON features.id = sell.features_id JOIN commune ON commune.id = sell.commune_id 
WHERE features.type_local = ? ORDER BY sell.valeur_fonciere DESC
 LIMIT ? OFFSET ? 



,valeur_fonciere,surface_carrez_du_1er_lot,code_departement,type_local
0,9000000.0,57.50,75,Appartement
1,8600000.0,74.13,91,Appartement
2,8577713.0,30.62,75,Appartement
3,7620000.0,14.31,75,Appartement
4,7600000.0,24.77,75,Appartement
5,7535000.0,53.44,75,Appartement
6,7420000.0,101.41,75,Appartement
7,7200000.0,35.42,75,Appartement
8,7050000.0,11.32,75,Appartement
9,6600000.0,76.61,75,Appartement


In [8]:
# 6: Rate of change in the number of sales 
# between the first and the second quarter of 2020

date = sell.c.date_mutation

first_sub_query = (session.query(func.count(sell.c.id)               
                                )
                   .filter(date.between("2020-01-01", 
                                        "2020-04-01"
                                       )
                          )
                   .scalar_subquery()
                  )

second_sub_query = (session.query(func.count(sell.c.id)
                                )
                   .filter(date.between("2020-04-01", 
                                        "2020-07-01"
                                       )
                          )
                    .scalar_subquery()
                  )

ratio = (100.0 * second_sub_query / first_sub_query - 100)

query = (session.query(first_sub_query.label("first_quarter"),
                       second_sub_query.label("second_quarter"),
                       ratio.label("Ratio second / first")
                      )      
        )

display(query_to_df(query))

Query : SELECT (SELECT count(sell.id) AS count_1 
FROM sell 
WHERE sell.date_mutation BETWEEN ? AND ?) AS first_quarter, (SELECT count(sell.id) AS count_2 
FROM sell 
WHERE sell.date_mutation BETWEEN ? AND ?) AS second_quarter, (? * (SELECT count(sell.id) AS count_2 
FROM sell 
WHERE sell.date_mutation BETWEEN ? AND ?)) / (SELECT count(sell.id) AS count_1 
FROM sell 
WHERE sell.date_mutation BETWEEN ? AND ?) - ? AS "Ratio second / first" 



,first_quarter,second_quarter,Ratio second / first
0,16774,17392,3.684273


In [9]:
# 7: List of communes where sale number rised about 20%
# between the first and the second quarter of 2020

first_query = (session.query(func.count(sell.c.id).label("first"),
                             commune.c.commune.label("commune")
                            )
               .filter(date.between("2020-01-01", 
                                    "2020-04-01"
                                   )
                      )
               .join(commune)
               .group_by(commune.c.commune)
               
              )

second_query = (session.query(func.count(sell.c.id).label("second"),
                              commune.c.commune.label("commune")
                             )
                .filter(date.between("2020-04-01", 
                                     "2020-07-01"
                                    )
                       )
                .join(commune)
                .group_by(commune.c.commune)
               ) 

sbq_1 = first_query.subquery()
sbq_2 = second_query.subquery()

ratio = 100 * (1.0 * sbq_2.c.second / sbq_1.c.first)

query = (session.query(sbq_1, 
                       sbq_2, 
                       func.round(ratio, 2).label("ratio_%"))
         .outerjoin(sbq_2, sbq_2.c.commune == sbq_1.c.commune)
         .filter(ratio >= 20)
        )
                      

display(query_to_df(query).fillna(0))

Query : SELECT anon_1.first AS anon_1_first, anon_1.commune AS anon_1_commune, anon_2.second AS anon_2_second, anon_2.commune AS anon_2_commune, round(? * ((? * anon_2.second) / anon_1.first), ?) AS "ratio_%" 
FROM (SELECT count(sell.id) AS first, commune.commune AS commune 
FROM sell JOIN commune ON commune.id = sell.commune_id 
WHERE sell.date_mutation BETWEEN ? AND ? GROUP BY commune.commune) AS anon_1 LEFT OUTER JOIN (SELECT count(sell.id) AS second, commune.commune AS commune 
FROM sell JOIN commune ON commune.id = sell.commune_id 
WHERE sell.date_mutation BETWEEN ? AND ? GROUP BY commune.commune) AS anon_2 ON anon_2.commune = anon_1.commune 
WHERE ? * ((? * anon_2.second) / anon_1.first) >= ? 



,first,commune,second,commune_1,ratio_%
0,12,ABLON-SUR-SEINE,17,ABLON-SUR-SEINE,141.67
1,1,ABRIES-RISTOLAS,3,ABRIES-RISTOLAS,300.00
2,3,ACHERES,4,ACHERES,133.33
3,6,AGDE,2,AGDE,33.33
4,2,AGEN,1,AGEN,50.00
...,...,...,...,...,...
1524,6,WISSANT,4,WISSANT,66.67
1525,11,WISSOUS,17,WISSOUS,154.55
1526,5,XONRUPT-LONGEMER,4,XONRUPT-LONGEMER,80.00
1527,4,YERRES,2,YERRES,50.00


In [10]:
# 8: Percent difference of price meter square 
# between appartement with 2 or 3 pieces


price_meter = sell.c.valeur_fonciere / features.c.surface_carrez_du_1er_lot

appartement_2 = (session.query(sell.c.valeur_fonciere,
                               features.c.surface_carrez_du_1er_lot,
                               price_meter.label("price_meter"),
                               features.c.type_local,
                               features.c.nombre_pieces_principales
                              )
                 .join(features)
                 .filter(
                     and_(
                         features.c.type_local == "Appartement",
                         features.c.nombre_pieces_principales == 2
                     )
                 )
                )

appartement_3 = (session.query(sell.c.valeur_fonciere,
                               features.c.surface_carrez_du_1er_lot,
                               price_meter.label("price_meter"),
                               features.c.type_local,
                               features.c.nombre_pieces_principales
                              )
                 .join(features)
                 .filter(
                     and_(
                         features.c.type_local == "Appartement",
                         features.c.nombre_pieces_principales == 3
                     )
                 )
                )

sbq_1 = appartement_2.subquery()
sbq_2 = appartement_3.subquery()

avg_2 = (session.query(func.avg(sbq_1.c.price_meter).label("avg_price_meter"),
                                ))

avg_3 = (session.query(func.avg(sbq_2.c.price_meter).label("avg_price_meter"),
                                ))

sbq_1 = avg_2.subquery()
sbq_2 = avg_3.subquery()

ratio = 100 * (1.0 * sbq_2.c.avg_price_meter / sbq_1.c.avg_price_meter)

query = (session.query(sbq_1.c.avg_price_meter.label("avg_price_meter_app_2"), 
                       sbq_2.c.avg_price_meter.label("avg_price_meter_app_3"), 
                       func.round(ratio, 2).label("ratio_%"))
         # .outerjoin(sbq_2, sbq_2.c.avg_price_meter == sbq_1.c.avg_price_meter)
        )

# display(query_to_df(appartement_2))
# display(query_to_df(appartement_3))

# display(query_to_df(avg_2))
# display(query_to_df(avg_3))

display(query_to_df(query))

Query : SELECT anon_1.avg_price_meter AS avg_price_meter_app_2, anon_2.avg_price_meter AS avg_price_meter_app_3, round(? * ((? * anon_2.avg_price_meter) / anon_1.avg_price_meter), ?) AS "ratio_%" 
FROM (SELECT avg(anon_3.price_meter) AS avg_price_meter 
FROM (SELECT sell.valeur_fonciere AS valeur_fonciere, features.surface_carrez_du_1er_lot AS surface_carrez_du_1er_lot, sell.valeur_fonciere / features.surface_carrez_du_1er_lot AS price_meter, features.type_local AS type_local, features.nombre_pieces_principales AS nombre_pieces_principales 
FROM sell JOIN features ON features.id = sell.features_id 
WHERE features.type_local = ? AND features.nombre_pieces_principales = ?) AS anon_3) AS anon_1, (SELECT avg(anon_4.price_meter) AS avg_price_meter 
FROM (SELECT sell.valeur_fonciere AS valeur_fonciere, features.surface_carrez_du_1er_lot AS surface_carrez_du_1er_lot, sell.valeur_fonciere / features.surface_carrez_du_1er_lot AS price_meter, features.type_local AS type_local, features.nombre_pi

C:\Users\arche\anaconda3\lib\site-packages\pandas\io\sql.py:1405: SAWarning: SELECT statement has a cartesian product between FROM element(s) "anon_2" and FROM element "anon_1".  Apply join condition(s) between each element to resolve.
  return self.connectable.execution_options().execute(*args, **kwargs)


,avg_price_meter_app_2,avg_price_meter_app_3,ratio_%
0,6527.772356,4273.930939,65.47


In [11]:
# 9: top 3 average valeur_fonciere of communes 
# in departements : 6, 13, 33, 59 and 69

dp_list = [6, 13, 33, 59, 69]

rank = (func.rank()
        .over(order_by=avg_valeur.desc(),
              partition_by=commune.c.code_departement
                      )
        .label("rank")
       )

avg_valeur = func.avg(sell.c.valeur_fonciere)

first_query = (session.query(avg_valeur.label("avg_valeur"),
                             commune.c.commune,
                             commune.c.code_departement,
                             rank
                            )
               .select_from(sell)
               .join(commune)
               .filter(commune.c.code_departement.in_(dp_list))
               .group_by(commune.c.commune)
              )

sbq = first_query.subquery()

query = (session.query(sbq)
         .filter(sbq.c.rank <= 3)
         .order_by(sbq.c.code_departement)
        )

display(query_to_df(query))

Query : SELECT anon_1.avg_valeur AS anon_1_avg_valeur, anon_1.commune AS anon_1_commune, anon_1.code_departement AS anon_1_code_departement, anon_1.rank AS anon_1_rank 
FROM (SELECT avg(sell.valeur_fonciere) AS avg_valeur, commune.commune AS commune, commune.code_departement AS code_departement, rank() OVER (PARTITION BY commune.code_departement ORDER BY avg(sell.valeur_fonciere) DESC) AS rank 
FROM sell JOIN commune ON commune.id = sell.commune_id 
WHERE commune.code_departement IN (__[POSTCOMPILE_code_departement_1]) GROUP BY commune.commune) AS anon_1 
WHERE anon_1.rank <= ? ORDER BY anon_1.code_departement 



,avg_valeur,commune,code_departement,rank
0,399500.000000,MARSEILLE 11EME,13,1
1,330000.000000,MARIGNANE,13,2
2,314425.000000,LA BOUILLADISSE,13,3
3,705000.000000,MARCHEPRIME,33,1
4,695051.000000,PAILLET,33,2
5,335000.000000,SAINT-CIERS-SUR-GIRONDE,33,3
6,433202.000000,HEM,59,1
7,408550.000000,TEMPLEUVE EN PEVELE,59,2
8,322250.000000,PERENCHIES,59,3
9,968750.000000,COLOMARS,6,1


In [12]:
# finally we close database connection
session.close()
engine.dispose()